In [1]:
!nvidia-smi -a -q -d CLOCK


==============NVSMI LOG==============

Timestamp                                 : Sat May 31 06:17:45 2025
Driver Version                            : 550.54.15
CUDA Version                              : 12.4

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Clocks
        Graphics                          : 300 MHz
        SM                                : 300 MHz
        Memory                            : 405 MHz
        Video                             : 540 MHz
    Applications Clocks
        Graphics                          : 585 MHz
        Memory                            : 5001 MHz
    Default Applications Clocks
        Graphics                          : 585 MHz
        Memory                            : 5001 MHz
    Deferred Clocks
        Memory                            : N/A
    Max Clocks
        Graphics                          : 1590 MHz
        SM                                : 1590 MHz
        Memory                            : 50

**Occupancy**

In [14]:
%%writefile one.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <iostream>

__global__ void add(int *results) {
  int gid = blockIdx.x * blockDim.x + threadIdx.x;

  int x1 = 1;
  int x2 = 2;
  int x3 = 3;
  int x4 = 4;
  int x5 = 5;
  int x6 = 6;
  int x7 = 7;
  int x8 = 8;

  results[gid] = x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8;
}

int main(){
  int *results;
  int size = 1;
  cudaMalloc(&results, size * sizeof(int));
  add<<<1,1>>>(results);
  cudaDeviceSynchronize();
}

Overwriting one.cu


In [15]:
!nvcc --ptxas-options=-v -o occupancy_test.out one.cu

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z3addPi' for 'sm_52'
ptxas info    : Function properties for _Z3addPi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 4 registers, 328 bytes cmem[0]


In [20]:
!ls

occupancy_test.out  one.cu  sample_data


In [22]:
!nvprof ./occupancy_test.out

==13636== NVPROF is profiling process 13636, command: ./occupancy_test.out
==13636== Profiling application: ./occupancy_test.out
==13636== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   64.86%  138.18ms         1  138.18ms  138.18ms  138.18ms  cudaMalloc
                   34.57%  73.654ms         1  73.654ms  73.654ms  73.654ms  cudaLaunchKernel
                    0.49%  1.0499ms         1  1.0499ms  1.0499ms  1.0499ms  cuDeviceGetPCIBusId
                    0.06%  132.16us       114  1.1590us     108ns  51.924us  cuDeviceGetAttribute
                    0.01%  11.484us         1  11.484us  11.484us  11.484us  cuDeviceGetName
                    0.01%  11.435us         1  11.435us  11.435us  11.435us  cudaDeviceSynchronize
                    0.00%  1.2660us         3     422ns     127ns     886ns  cuDeviceGetCount
                    0.00%     920ns         2     460ns     151ns     76